#  Exercise  for using  low level functions in Tensorflow

The idea of this exercise is to import data, create train/test datasets, implement a neural network , and train it. TensorFlow includes nice functions to implement many functions in high abtraction level. It is sometimes a good exercise to do more things manually to gain more understanding  what really happens  inside the neural network. Furthermore, if you would like to develop your own functions, such as more advanced gradient descent, you need to be able to do things under the hood.


Let us start by first downloading the data.  We use here MNIST dataset and we download it from the Yann LeCun webpage (http://yann.lecun.com/exdb/mnist/).  Download the files "train-images-idx3-ubyte.gz" and "train-labels-idx1-ubyte.gz" in your computer into your working folder and unzip the files if your browser did not do it automatically.

Next, we do all the needed imports. We need numpy, matplotlib, pylab and tensorflow. The MNIST data is in the IDX file format, we use a tool called "idx2numpy" for importing the data.  The idx2numpy imports the data directly to a numpy array. You can install it with pip install idx2numpy. The "%matplotlib inline" allows you to print directly in this notebook.

In [1]:
# do all the needed imports; numpy, matplotlib, tensorflow, pylab, idx2numpy
import numpy as np
import matplotlib.pyplot as plt
import pylab
import idx2numpy
import tensorflow as tf
%matplotlib inline

The first task is to import the data that you have previously downloaded into numpy arrays. If you installed the idx2numpy package, you can use idx2numpy.convert_from_file to import the data.  Import the actual MNIST images and labels into separate numpy arrays.

In [2]:
# open the file for reading
# read the data into a numpy array
# close the file


Let us construct the training and test sets.  For this exercise, we do not need all the data. Let us take the first 30000 images from the original training set as our training set, and the following 3000 samples from the original training set as our test set.  For this exercise, we do not need a validation set.

In [3]:
# take the samples 1-30000 as a training set
# take the samples 30001-33000 as a test set

X_train = ???
y_train_labels = ???
X_test = ???
y_test_labels = ???

Let us investigate the data. You can observe, that the data values are not between 0 and 1, which might be good for our purposes. Try to investigate what is the range of the values and normalize the data so that the pixel values are between 0 and 1.  

In [4]:
# normalize the pixel value to be between 0 and 1

X_train = ???
X_test = ???

Now we can plot the data with  e.g. pyplot.  Create a function to plot the first 25 MNIST digits in a nice 5 x 5 picture grid.

In [1]:
# plot the first 26 MNISt images into 5 x 5 grid



The lables are now in a single list of numbers 0-9. For the cross-entropy loss calculations, the labels need to be in onehot format (can you say why?). Construct a simple function to convert  the labels into one-hot format. Make sure your function works with different batch sizes, including a batch size of one.

In [6]:
# construct a function to convert list of labels to onehot format

def onehot(input):
    ???
    return onehot

In [7]:
# use the function to create the onehot vectors for both the training set and the test set
y_train_onehot=onehot(y_train_labels)
y_test_onehot=onehot(y_test_labels)

Now we should be ready to start constructing a simple MLP network that we can use for our experiments. We will now take just 300 samples from our training set to keep our computations fast at this point. We will first construct a simple MLP, which takes one vector as an input, so we flatten each MNIST image into one single row vector. All images will be in a single matrix, where the rows correspond to the number of images in the batch size, and the columns represent the input dimensionality. One flattened 28x28 pixel MNIST image has thus shape 1 x 784.

In [8]:
# take 300 samples from the training set
# flatten each 28x28 image into single row vector and construct the input matrix 

X_in=X_train[0:300].reshape(-1,784)
y_in=y_train_onehot[0:300]

Construct a tensorflow computational graph for an as simple as possible linear mlp network. Only input and output layer and softmax activation. Use cross entropy loss. 

In [9]:
#Placeholders
x = ???
y_ = ???

In [10]:
# Weights and biases
W1 = ???
b1 = ???

In [11]:
#Tensorflow computational graph 
y = ???

In [12]:
#loss and training step
loss = ???


In [13]:
#session intialization
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [ ]:
learning_rate = 0.5
accuracy_values = np.array([])
cost_values = np.array([])
var_list = [W1, b1]

for i in range(50):
    #sess.run(train_step, feed_dict={x: X_in, y_: y_in})
    loss_train = loss.eval(feed_dict={x: X_in, y_: y_in})
    cost_values = np.append(cost_values, loss_train)

    gradients = ??? #calculate gradients
      
    updates = ??? # update gradients
    
    gradient_descent = tf.group(*updates) # control flow op for simultaneous updates

    sess.run(gradient_descent, feed_dict={x: X_in, y_: y_in})
    
    if i % 10 == 0:        
        print 'iteration {}: loss: {}'.format(i, loss_train)


# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print'training accuracy',sess.run(accuracy, feed_dict={x: X_in, y_: y_in})
print'test accuracy',sess.run(accuracy, feed_dict={x: X_test.reshape(-1,784), y_: y_test_onehot})

plt.plot(cost_values)

Let us next try to construct a simple vanilla recurrent neural network in a similar manner. It is a good idea to define all hyperparameters in the beginning to make the program more scalable.

In [15]:
input_xdim = 28
input_ydim = 28
learning_rate = 1e-4
hidden_features = 100
batch_size = 100
classes = 10
iterations = 1000 # Must be at least 10k to obtain over 90%

Let's define the placeholders for the input data. This time the placeholder for the input image needs to be a tensor of rank 3, as we will feed one row of the image at each timestep.

In [16]:
input_rows = tf.placeholder(tf.float32, [None, input_xdim, input_ydim])
labels = tf.placeholder(tf.float32, [None, classes])

Next we want to define the trainable variables. In each RNN cell we need one weight matrix W for the hidden vector passed on from the previous cell, one weight matrix U for the actual input to the cell and a bias b1. We will also need a weight matrix V for the output layer from each cell and a bias b2.

In [17]:
W = tf.Variable(tf.truncated_normal([hidden_features, hidden_features], stddev=0.1))
U = tf.Variable(tf.truncated_normal([input_xdim, hidden_features], stddev=0.1))
V = tf.Variable(tf.truncated_normal([hidden_features, classes], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[1, hidden_features]))
b2 = tf.Variable(tf.constant(0.1, shape=[1, classes]))

It is a good idea to have a function available for constructing the output layers. So let's define a function that takes as parameters an input and an activation function. If the function is called to create an output layer, it can be called with softmax as the activation. Use the above defined variables V and b2.

In [18]:
def nn_layer(inputs, activation):
    outputs = ???
    return outputs

Now we make a very simple function for creating a vanilla RNN cell. It takes as parameters a particular row of the image and a hidden state. The output should be the new hidden state. Use the above defined variables W, U and b1, and tanh as activation.

In [19]:
def rnn_cell(inputs, hidden):
    hidden_out = ???
    return hidden_out

In the next function we implement the actual RNN network. There is one RNN cell for each row, and each cell outputs a hidden vector, and an output state which is the hidden vector passed through a linear output layer with softmax. Average across all softmax outputs to get final prediction. There are many ways to do it, feel free to try out your own implementation. You can alternatively try to fill in the two incomplete parts in the function below. The only thing you have to do is to call the two previous functions with the right parameters!

In [20]:
def rnn(inputs):
    rows = tf.unstack(inputs, input_ydim, 1) # Make an array of 2D tensors
    samples = tf.shape(rows)[1] # Number of samples in the batch/test set
    hidden_vec = tf.zeros([samples, hidden_features]) # Here we will store the hidden output from each cell
    for cell_index in range(input_ydim): # Iterate through all cells
        current = rows[cell_index] # Get the current row
        hidden_vec = ??? # Compute the hidden output from the current cell 
        cell_output = ??? # Compute the final output from the current cell
        if cell_index == 0:
            out = cell_output
        else:
            out = out + cell_output # Add together final outputs of all cells
    return tf.multiply(1.0/input_ydim, out) # and divide to get average

Now we are ready to get the prediction and calculate the cost function. For the prediction, we just call the rnn function and for the cost function write your own cross entropy function, i.e. multiply onehot label with the negative logarithm of the prediction, sum over all classes and average over all samples in the batch.

In [29]:
score = rnn(input_rows)
cost_function = ???

In [30]:
grads = tf.gradients(cost_function, [V, b2, U, W, b1])

Next try to implement your own version of gradient descent. tf.assign takes as the first parameter the variable to be updated as a second parameter the updated variable. The updated variable is simply the original variable minus learning rate multiplied by the gradient for the respective variable.

In [31]:
updates = [tf.assign(???),
                   tf.assign(???),
                   tf.assign(???),
                   tf.assign(???),
                   tf.assign(???)]

gradient_descent = tf.group(*updates)

In [32]:
true_labels = tf.argmax(labels, axis = 1)
predicted_labels = tf.argmax(score, 1)

In [33]:
evaluate = tf.equal(true_labels, predicted_labels)
accuracy = tf.reduce_mean(tf.cast(evaluate, tf.float32))

In [34]:
train_image = X_train.reshape(30000, input_ydim, input_xdim)
test_image = X_test.reshape(3000, input_ydim, input_xdim)

In [ ]:
train_loss = np.zeros(iterations/100, float)
train_acc = np.zeros(iterations/100, float)

In [35]:
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(iterations):
    start = (i * batch_size) % (30000 - (2 * batch_size))
    end = start + batch_size
    batch_image = X_train[start:end, :, :]
    batch_label = y_train_onehot[start:end, :]
    sess.run(gradient_descent, feed_dict={input_rows: batch_image, labels: batch_label})
    if i % 100 == 0:  
        print 'iteration {}: '.format(i)
print("The test accuracy is %g" %accuracy.eval(feed_dict={input_rows: test_image, labels: y_test_onehot}))

The final part of the exercise is to visualize the results for the rnn network. We will plot the training loss and accuracy, and draw a confusion matrix to see how the true and predicted classes relate to one another. (The main goal here is to practise visulizations with matplotlib, keep in mind that rnn's are not ideal for learning mnist.)

In [ ]:
confusion_matrix = ??? # use tf.confusion_matrix
cm = confusion_matrix.eval(feed_dict={input_rows: test_image, labels: y_test_onehot})

In [ ]:
fig1 = plt.figure()
iters = range(len(train_loss)) # x values values for plots
# Use matplotlib to plot training cost and accuracy
???
plt.draw()

fig2 = plt.figure()
# Use matplotlib to draw confusion matrix
???
plt.draw()
sess.close()
plt.show()